<img src="img/Brilliant_logo.png" width="20%">

### John Mahoney's Teaching Demo

Sept 8, 2020

mohnjahoney@gmail.com

mohnjahoney@github.io

TODOs:
- Get this working on Binder. DONE!
- nbinteract + bqplot interactive static webpage? NOT NEEDED
- Use font / bootstrap? to highlight main lessons and quiz questions / answers.
- Make a nice format for when we pose an important question.
- Make a nice format for when we present an important result / summary point.
- Nicer Quiz and Q/A formatting. Bootstrap options?
- Hide quiz answer until hover? (i think this exists)
- Turn notebook into slides - start with a simple example.
- Interactive table. You put in numbers to try to make a Simpson - and maybe while meeting certain constraints. This removes the mental math (but still not geometric). Then maybe add the geometric part. ? Link graph to table?
- 
- 

# Lessons

- Simpson's paradox has something to do with aggregation and things switching.

- Be careful when you add: are you adding **numbers** (stuff) or **ratios** (not stuff)?

Example: Baseball batting averages can switch when we aggregate. However, **runs** will not switch.

- There are limits:
        you can't combine (batting aves) {0.1, 0.2, 0.3} and overcome {0.6, 0.7, 0.8} by aggregating - there has to be some overlap.
        
- What if drug A is better than B for women and men, but B is better for people as a group? 

- People seem to be comfortable with the idea that if 2019 BA was x and 2020 was y, then the aggregate should be between x and y. (and this is true)
- People seem (fairly) comfortable with weighted averages. If there were many more at bats in 2019, then the aggregate should be closer to x than to y.
- If we have two batter and the at bat distribution is the same for 2019 and 2020, then there can be no SP.
- If the two batters have overlapping BAs, and different yearly at bats, and the "lower" one gets pulled up and the "higher" one gets pulled down, the result can be a flip.

- OK, but what should you *do*?

Take the example of males and females under the old and new education plans.
The new plan increases scores of both groups, but the overall average score is lower.
So what should you do next year? use the new plan or not?

In [1]:
# Imports
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
#import matplotlib.pyplot as plt
import bqplot as bq
import bqplot.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LinearRegression

# Simpson's Paradox

Some ideas in statistics have an intuitive feel to them, even if they might take some work to compute precisely.
For example, we can intuitively understand that the **mean** (or the average) of a set of numbers is just *the middle*.
Using our intuition, we might guess that the mean of ${2,4,7,9}$ is near 5 or 6 and the mean of ${23, 55, 92}$ is probably close to 50.

Some other statistical ideas are more subtle.
To build intuition, we'll have to dig in!

In this lesson, we'll be exploring an idea known as **Simpson's Paradox**.

We call this a paradox because some of the situations below might contradict your current intuition.

Our goals in this lesson are to:
- notice how our intuition may be wrong about some simple situations,
- gain some intuition about the paradox so that it feels... less paradoxical,
- become aware of the different flavors of this paradox,
- understand how to act when faced with this paradox in real life.

# QUIZ: 

Which of the following *best* characterizes Simpson's paradox?

- A dataset may display both a positive and negative trend at the same time.
- A trend changes sign upon addition of just a single datapoint.
- A trend changes sign when data are combined. (YES)

## The Gist

**Combining subsets of data can result in counter-intuitive reversals.**

CUT THIS
One one hand, Simpson's paradox can seem rather, well.., paradoxical.
On the other, its essence is not complicated.

The main difficulty arises mostly because people are not that good at mental math.
Also, there are different flavors of Simpson's paradox and this can be confusing too.
END CUT

Let's explore three examples to get a better feel for this paradox and its implications.


# Example 1: The shape of fish.

Here we see two groups of fish, some red and some blue.
There are all different sizes.
We can see the the red fish are generally "tall" while the blue fish are generally "long".

In [2]:
# Create fish data
fish = pd.DataFrame({'color':['red', 'red', 'red', 'blue', 'blue', 'blue'], 
                     'length':[0.5, 1.0, 1.5, 2.0, 3.0, 4.0], 
                     'height':[2.5, 3.5, 4.5, 1.0, 1.5, 2.0]})
# fish = pd.DataFrame({'color':['red', 'red', 'red', 'blue', 'blue', 'blue'], 
#                      'length':[0.5, 1.0, 1.5, 1.0, 2.5, 4.0], 
#                      'height':[2.5, 3.5, 4.5, 1.0, 1.5, 2.0]})

# We don't want the image xspan to be as big as the length coordinate - otherwise the fish will overlap too much.
x_scale = 0.5
y_scale = 0.5

fish['xspan'] = x_scale * fish['length']
fish['yspan'] = y_scale * fish['height']

# We want the fish to start out somewhere "random".
fish['init_x'] = [4.2, 1.5, 3.9, 0.5, 3.1, 1.3]
fish['init_y'] = [0.6, 3.8, 4.4, 4.5, 2.3, 1]

# fish

In [3]:
# TODO: Do I have to define the scales before defining the images??
x_sc = bq.LinearScale(min=0, max=5)
y_sc = bq.LinearScale(min=0, max=6)

def get_image(color):
    if color == 'red':
        image = bq.Image(image=ipyimageA, scales={'x':x_sc, 'y':y_sc})
    elif color == 'blue':
        image = bq.Image(image=ipyimageB, scales={'x':x_sc, 'y':y_sc})
    else:
        raise
    return image


def scale_and_place_image(row, init=False):

    if init:
        x = row['init_x']
        y = row['init_y']
    else:
        x = row['length']
        y = row['height']

    # NOTE: The extent of the image does not depend on whether it is in "initial" or "normal" position.
    dx = row['xspan']
    dy = row['yspan']
    
    image = row['image']
    image.x = [x - dx/2, x + dx/2]
    image.y = [y - dy/2, y + dy/2]

    return image

In [4]:
# Put images in the dataframe

# Fish images
image_pathA = os.path.abspath('./img/fish_red.png')
image_pathB = os.path.abspath('./img/fish_blue.png')

with open(image_pathA, 'rb') as fA:
    raw_imageA = fA.read()
with open(image_pathB, 'rb') as fB:
    raw_imageB = fB.read()
    
ipyimageA = widgets.Image(value=raw_imageA, format='png')
ipyimageB = widgets.Image(value=raw_imageB, format='png')

# Get the right color fish.
fish['image'] = fish['color'].apply(get_image)
# Stretch each fish in some way.
fish['image'] = fish.apply(lambda row: scale_and_place_image(row, init=True), axis=1)

In [5]:
# Make initial "ocean" plot.
    
ocean_xs = np.linspace(0, 5, 400)
ocean_ys = -0.4 * np.abs(np.sin(4*ocean_xs)) + 6.0

x_ax = bq.Axis(label='length', scale=x_sc)
y_ax = bq.Axis(label='height', scale=y_sc, orientation='vertical')

ocean_line = plt.plot(ocean_xs, ocean_ys, scales={'x':x_sc, 'y':y_sc})

marks = list(fish['image']) + [ocean_line]

fig = bq.Figure(marks=marks, axes=[], animation_duration=1000, 
                layout=widgets.Layout(width='auto'))
# fig = bq.Figure(marks=marks, axes=[x_ax, y_ax], animation_duration=1000, 
#                 layout=widgets.Layout(width='50%'))

box = widgets.GridspecLayout(1, 1, layout=widgets.Layout(border='blue solid 1px', width='50%'))
box[0, 0] = fig

box

GridspecLayout(children=(Figure(animation_duration=1000, fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'rig…

## Visualize the data.
After measuring the dimensions of each fish, let's organize them according to their length and height.

In [6]:
# Move fish into (length, height) position on respective figures.

# TODO: Use separate scales for the two different plots to make the aggregate relation less obvious?
# TODO: Axis labels
# TODO: Fix layout: wider, less margin
# TODO: Dot size

# Move the fish into their "correct" places.
fish['image'] = fish.apply(lambda row: scale_and_place_image(row, init=False), axis=1)

# Mark with center dots for clarity.
scatterA = bq.Scatter(x=fish[fish['color']=='red']['length'], y=fish[fish['color']=='red']['height'], 
                      colors=['Red'], scales={'x': x_sc, 'y': y_sc})
scatterB = bq.Scatter(x=fish[fish['color']=='blue']['length'], y=fish[fish['color']=='blue']['height'], 
                      colors=['Blue'], scales={'x': x_sc, 'y': y_sc})

# Put things into figures.
marksA = list(fish[fish['color']=='red']['image']) + [scatterA]
marksB = list(fish[fish['color']=='blue']['image']) + [scatterB]

# figA = bq.Figure(marks=marksA, axes=[x_ax, y_ax], animation_duration=1000, 
#                     min_aspect_ratio=1, max_aspect_ratio=1, layout=widgets.Layout(border='red solid 2px', length='auto'))
# figB = bq.Figure(marks=marksB, axes=[x_ax, y_ax], animation_duration=1000, 
#                      min_aspect_ratio=1, max_aspect_ratio=1, layout=widgets.Layout(border='red solid 2px'))

figA = bq.Figure(title='Red Fish', marks=marksA, axes=[x_ax, y_ax], animation_duration=1000, 
                 layout=widgets.Layout(border='red solid 1px', width='auto'))
figB = bq.Figure(title='Blue Fish', marks=marksB, axes=[x_ax, y_ax], animation_duration=1000, 
                 layout=widgets.Layout(border='red solid 1px', width='auto'))

box = widgets.GridspecLayout(1, 2, layout=widgets.Layout(border='blue solid 1px', width='100%'))
box[0, 0] = figA
box[0, 1] = figB

box

GridspecLayout(children=(Figure(animation_duration=1000, axes=[Axis(label='length', scale=LinearScale(max=5.0,…

# TODO: turn into multiple choice

## Quiz! Read and interpret the graph.

Try answering a few questions based on the graphs above.

- Imagine you find a blue fish that is 4 foot long. How tall should you expect it to be?
(answer: 2 feet)

- Imagine you find a red fish that is 2 feet long. How tall should you expect it to be?
(answer: 5.5 feet)

## Observe correlations.

You have probably observed some trends or predictability in this data.

For example, focus on the red fish: We see that as the fish get longer, they also get taller.
In other words, the length and height are **positively correlated**.

Now focus on the blue fish: We see a similar trend - as the fish get longer, they get taller too.
The length and height of these blue fish are **also positively correlated**.

We can visualize these correlations by drawing a line through each group.
The slope of each line tells us how much the height will change for a given change in length.

$$\textrm{slope} = \frac{\Delta y}{\Delta x} = \frac{\textrm{difference in fish height}}{\text{difference in fish length}}$$

In [7]:
# Add individual regression lines.

# Note: Using regression here for consistency with the aggregate and also to be more flexible.


regA = LinearRegression().fit(fish[['length']][fish['color']=='red'],
                              fish[['height']][fish['color']=='red'])
regB = LinearRegression().fit(fish[['length']][fish['color']=='blue'],
                              fish[['height']][fish['color']=='blue'])

reg_xs = np.linspace(0, 5, 2).reshape(-1, 1)

reg_ysA = regA.predict(reg_xs)
reg_ysB = regB.predict(reg_xs)

line_regA = bq.Lines(x=reg_xs, y=reg_ysA, colors=['Pink'], scales={'x': x_sc, 'y': y_sc})
line_regB = bq.Lines(x=reg_xs, y=reg_ysB, colors=['lightblue'], scales={'x': x_sc, 'y': y_sc})

figA.set_trait('marks', figA.marks + [line_regA])
figB.set_trait('marks', figB.marks + [line_regB])

# TODO: turn into multiple choice

## Quiz! Understand correlation.

- Red fish: When the length increases by one foot, the height (increases / decreases) by ___ feet.
(answer: increases, 2)

- Blue fish: When the length increases by one foot, the height (increases / decreases) by ___ feet.
(answer: increases, 1/3)

# make the question less boring

## Quiz: Label each example "positive" or "negative" depending on the correlation.
- Number of tacos eaten for lunch : Number of calories consumed for lunch (+)
- Number of tacos eaten for lunch : Amount of money remaining in wallet (-)
- Spiciness of salsa : Amount of salsa used (-)
- Put in a plot or two?

## Things get confusing.

We've seen that, within the group of red fish, longer fish are taller.
The same is true for blue fish.

What happens when we combine these two groups of fish and consider **all** fish together?
It seems only natural that the same trend should hold for the collection of red and blue fish.

Let's see what happens!...

In [8]:
# Aggregate the two groups of fish. Draw an aggregate regression line.

# TODO: If we split the figures up front, here we would merge into the same plot.

regAB = LinearRegression().fit(fish[['length']],
                              fish[['height']])

reg_ysAB = regAB.predict(reg_xs)

line_regAB = bq.Lines(x=reg_xs, y=reg_ysAB, colors=['black'], scales={'x': x_sc, 'y': y_sc})

marksAB = marksA + marksB + [line_regA, line_regB]

figAB = bq.Figure(title='All Fish', marks=marksAB, axes=[x_ax, y_ax], animation_duration=1000, 
                 layout=widgets.Layout(border='white solid 2px', width='auto'))

###

scatterA2 = bq.Scatter(x=fish[fish['color']=='red']['length'], y=fish[fish['color']=='red']['height'], 
                      colors=['green'], scales={'x': x_sc, 'y': y_sc})
scatterB2 = bq.Scatter(x=fish[fish['color']=='blue']['length'], y=fish[fish['color']=='blue']['height'], 
                      colors=['green'], scales={'x': x_sc, 'y': y_sc})

marksAB_BW = [scatterA2, scatterB2, line_regAB]

figAB_BW = bq.Figure(title='All Fish', marks=marksAB_BW, axes=[x_ax, y_ax], animation_duration=1000, 
                 layout=widgets.Layout(border='white solid 2px', width='auto'))
     
boxAB = widgets.GridspecLayout(1, 2)
boxAB[0, 0] = figAB
boxAB[0, 1] = figAB_BW

boxAB

GridspecLayout(children=(Figure(animation_duration=1000, axes=[Axis(label='length', scale=LinearScale(max=5.0,…

Surprisingly, the trend is actually reversed!

In combining the two groups of fish we find that the length and height are now **negatively correlated**.

Let's say that clearly:

> The correlation between length and height is positive within each subgroup of fish. However, this correlation is negative within the entire group of fish.

This is the essence of Simpson's Paradox.

## Interactive challenge!

Can you create a Simpson's Paradox situation with **three** subgroups?
Move the shapes (circles, squares, and triangles) so that the correlation is **negative within each group**, yet the **overall trend is positive**.

In [9]:
# Play the game!

# TODO: Add Play Again reset button

xmax = 10
ymax = 9

# xs = np.random.uniform(0, xmax, 9)
# ys = np.random.uniform(0, ymax, 9)
xs = np.array([1.0, 4.0, 6.0, 4, 5, 8, 3, 5, 9])
ys = np.array([8.0, 6.0, 9.0, 2, 1, 4, 1, 4, 3])

circle_inds = [0, 1, 2]
square_inds = [3, 4, 5]
triangle_inds = [6, 7, 8]

fig = plt.figure()
# x_sc = bq.LinearScale(min=0, max=7)
# y_sc = bq.LinearScale(min=0, max=8)

reg0 = LinearRegression().fit(xs[circle_inds].reshape(-1, 1), ys[circle_inds])
reg1 = LinearRegression().fit(xs[square_inds].reshape(-1, 1), ys[square_inds])
reg2 = LinearRegression().fit(xs[triangle_inds].reshape(-1, 1), ys[triangle_inds])
reg012 = LinearRegression().fit(xs.reshape(-1, 1), ys)

reg_xs = np.linspace(0, 10, 2)

reg_ys0 = reg0.predict(reg_xs.reshape(-1, 1))
reg_ys1 = reg1.predict(reg_xs.reshape(-1, 1))
reg_ys2 = reg2.predict(reg_xs.reshape(-1, 1))
reg_ys012 = reg012.predict(reg_xs.reshape(-1, 1))

line_reg0 = plt.plot(reg_xs, reg_ys0, colors=['green'], stroke_width=3)
line_reg1 = plt.plot(reg_xs, reg_ys1, colors=['green'], stroke_width=3)
line_reg2 = plt.plot(reg_xs, reg_ys2, colors=['green'], stroke_width=3)
line_reg012 = plt.plot(reg_xs, reg_ys012, colors=['red'], stroke_width=6, line_style='dashed')

scatter0 = plt.scatter(xs[circle_inds], ys[circle_inds], default_size=600, colors=['gray'], 
                       marker='circle', enable_move=True)
scatter1 = plt.scatter(xs[square_inds], ys[square_inds], default_size=600, colors=['gray'], 
                       marker='square', enable_move=True)
scatter2 = plt.scatter(xs[triangle_inds], ys[triangle_inds], default_size=600, colors=['gray'], 
                       marker='triangle-up', enable_move=True)

status_text = plt.label(["Not Yet"], x=[5], y=[11], 
                            align='middle', font_weight='bold', default_size=24, colors=['Black'])

def randomize_positions(button, scatter0, scatter1, scatter2):
#    print("wer")
    scatter0.x = np.random.uniform(0, xmax, 3)
    scatter1.x = np.random.uniform(0, xmax, 3)
    scatter2.x = np.random.uniform(0, xmax, 3)
    scatter0.y = np.random.uniform(0, ymax, 3)
    scatter1.y = np.random.uniform(0, ymax, 3)
    scatter2.y = np.random.uniform(0, ymax, 3)
    
button_randomize = widgets.Button(
    description='Randomize positions',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description')
    
button_randomize.on_click(lambda button: randomize_positions(button, scatter0, scatter1, scatter2))

def is_simpson(reg0, reg1, reg2, reg012, status_text):
    conds = [(reg0.coef_ < 0), (reg1.coef_ < 0), (reg2.coef_ < 0), (reg012.coef_ > 0)]
    if conds[0]:
        line_reg0.set_trait('opacities', [1])
    else:
        line_reg0.set_trait('opacities', [0.2])
        
    if conds[1]:
        line_reg1.set_trait('opacities', [1])
    else:
        line_reg1.set_trait('opacities', [0.2])
        
    if conds[2]:
        line_reg2.set_trait('opacities', [1])
    else:
        line_reg2.set_trait('opacities', [0.2])
        
    if conds[3]:
        line_reg012.set_trait('opacities', [1])
    else:
        line_reg012.set_trait('opacities', [0.2])
      
    if sum(conds[:-1]) == 0:
        status_text.text = ["Move the shapes"]
    elif sum(conds[:-1]) == 1:
        status_text.text = ["You got one :)"]
    elif sum(conds[:-1]) == 2:
        status_text.text = ["Even better!"]
    elif sum(conds[:-1]) == 3 and conds[-1] == 0:
        status_text.text = ["Now for the total!"]
    elif sum(conds) == 4:
        status_text.text = ["You designed a Simpson's Paradox!"]
    else:
        print('ELSE+++++++++++++++++++++++')
        
def update_reg_line(change):
    reg0 = LinearRegression().fit(scatter0.x.reshape(-1, 1), scatter0.y.reshape(-1, 1))
    reg1 = LinearRegression().fit(scatter1.x.reshape(-1, 1), scatter1.y.reshape(-1, 1))
    reg2 = LinearRegression().fit(scatter2.x.reshape(-1, 1), scatter2.y.reshape(-1, 1))
    
    newxs = np.concatenate([scatter0.x, scatter1.x, scatter2.x])
    newys = np.concatenate([scatter0.y, scatter1.y, scatter2.y])
    reg012 = LinearRegression().fit(newxs.reshape(-1, 1), newys.reshape(-1, 1))
    
    reg_ys0 = reg0.predict(reg_xs.reshape(-1, 1))
    reg_ys1 = reg1.predict(reg_xs.reshape(-1, 1))
    reg_ys2 = reg2.predict(reg_xs.reshape(-1, 1))
    reg_ys012 = reg012.predict(reg_xs.reshape(-1, 1))

    line_reg0.y = reg_ys0
    line_reg1.y = reg_ys1
    line_reg2.y = reg_ys2
    line_reg012.y = reg_ys012
    
    is_simpson(reg0, reg1, reg2, reg012, status_text)
    
scatter0.observe(update_reg_line, names=['x','y'])
scatter1.observe(update_reg_line, names=['x','y'])
scatter2.observe(update_reg_line, names=['x','y'])

is_simpson(reg0, reg1, reg2, reg012, status_text)
    
plt.xlim(0, xmax)
plt.ylim(0, 11)

fig.layout = widgets.Layout(width='100%')

fig.marks = fig.marks + [status_text]
fig.axes[0].set_trait('visible', False)
fig.axes[1].set_trait('visible', False)

box = widgets.GridspecLayout(3, 4, layout=widgets.Layout(width='80%'))
box[:, 0:3] = fig
box[1, 3] = button_randomize

box

GridspecLayout(children=(Figure(axes=[Axis(scale=LinearScale(max=10.0, min=0.0), visible=False), Axis(orientat…

In [10]:
%%HTML
<style>
td {
  font-size: 16px
}
</style>
# Adjust the font size of the table.
# TODO: Check that this does not mess with other sizes in the doc. Is there another way?

In [11]:
# Create batting dataframe

# Set up multi index dataframe for batting stats
years = [2018, 2019]
batting_details = ['hits', 'at-bats', 'BA']

mindex = pd.MultiIndex.from_product([years, batting_details],
                           names=['year', 'stats'])

# This will set dtypes as int. Batting ave will upcast to float later.
bat_df = pd.DataFrame(np.zeros(shape=(2, 6), dtype='int'), index=['Jack', 'Arlo'], columns=mindex)

########
# Enter the main data: 'hits' and 'at bats'
bat_df.loc['Jack'] = [1, 20, 0.0, 24, 80, 0.0]
bat_df.loc['Arlo'] = [12, 80, 0.0, 10, 20, 0.0]
########

# TODO: This could be done better.
bat_df.loc[:, ('Total', 'hits')] = bat_df.loc[:, (2018, 'hits')] + bat_df.loc[:, (2019, 'hits')]
bat_df.loc[:, ('Total', 'at-bats')] = bat_df.loc[:, (2018, 'at-bats')] + bat_df.loc[:, (2019, 'at-bats')]

def compute_BA(df):
    for year in df.columns.get_level_values('year').unique():
        df.loc[:, (year,'BA')] = df[year]['hits'] / df[year]['at-bats']
    return df

bat_df = compute_BA(bat_df)

# Example 2: Batting averages (BA)

Jack and Arlo are two baseball players - here are their runs and at-bats for the past two years.

{{ bat_df }}

Baseball fans argue about which one is the better player.

Some fans say "Arlo is definitely the better player! He had a better batting average in each of the two seasons.

(Take a minute and confirm that this is true.)

Other fans respond "Yeah, but Jack is clearly the better player *overall*."

(Confirm that Jack has a higher cumulative batting average.)

This is a classic example of Simpson's paradox.

First, let's try to understand more about how this situation can arise.
Then, we'll come back to the fans and try to help settle their debate.

In [12]:
# slice(None) important for slicing a multi index
hits = bat_df.loc[:, (slice(None), 'hits')].to_numpy()
atbats = bat_df.loc[:, (slice(None), 'at-bats')].to_numpy()

# We want to plot the cumulative.
# -1 because we don't want the Total
hitscum = np.cumsum(hits[:, :-1], axis=1)
atbatscum = np.cumsum(atbats[:, :-1], axis=1)

# Prepend 0s for plotting
hitscum = np.hstack((np.zeros(shape=(2,1)), hitscum))
atbatscum = np.hstack((np.zeros(shape=(2,1)), atbatscum))

#print(hitscum)
#print(atbatscum)

In [13]:
# Color choices
jack_colors = ['red', 'red']
arlo_colors = ['blue', 'blue']

# jack_colors = ['red', 'blue']
# arlo_colors = ['red', 'blue']

# jack_colors = ['red', 'orange']
# arlo_colors = ['blue', 'green']

In [14]:
# Make batting average interactive!

x_sc2 = bq.LinearScale(min=0, max=110)
y_sc2 = bq.LinearScale(min=0, max=27)

x_ax2 = bq.Axis(label='at-bats', scale=x_sc2)
y_ax2 = bq.Axis(label='runs', scale=y_sc2, orientation='vertical')


jack_lines = [bq.Lines(x=atbatscum[0, [ind, ind+1]], y=hitscum[0, [ind, ind+1]], colors=[jack_colors[ind]], 
                       stroke_width=5, line_style='solid', scales={'x': x_sc2, 'y': y_sc2}) for ind in range(2)]
arlo_lines = [bq.Lines(x=atbatscum[1, [ind, ind+1]], y=hitscum[1, [ind, ind+1]], colors=[arlo_colors[ind]], 
                       stroke_width=5, line_style='solid', scales={'x': x_sc2, 'y': y_sc2}) for ind in range(2)]

jack_lines[1].set_trait('line_style', 'dashed')
arlo_lines[1].set_trait('line_style', 'dashed')
jack_lines[0].set_trait('labels', [''])
arlo_lines[0].set_trait('labels', [''])
jack_lines[1].set_trait('labels', ['Jack'])
arlo_lines[1].set_trait('labels', ['Arlo'])

vert_lines = [bq.Lines(x=2*[atbatscum[ind, 1]], y=[0, 25], colors=['black'], line_style='solid', 
                       stroke_width=1, scales={'x': x_sc2, 'y': y_sc2}) for ind in [0, 1]]

jack_text = plt.label(["Jack"], x=[35], y=[20], 
                            align='middle', font_weight='bold', default_size=24, colors=['Red'], scales={'x': x_sc2, 'y': y_sc2})
arlo_text = plt.label(["Arlo"], x=[65], y=[20], 
                            align='middle', font_weight='bold', default_size=24, colors=['Blue'], scales={'x': x_sc2, 'y': y_sc2})

fig_batting = plt.figure(marks=vert_lines + jack_lines + arlo_lines + [jack_text, arlo_text], axes=[x_ax2, y_ax2], 
                        layout=widgets.Layout(width='auto', height='auto'), animation_duration=1000, 
                        fig_margin={'top':20, 'bottom':50, 'left':50, 'right':20}, display_legend=True)

button_bat1 = widgets.ToggleButtons(
    value='season',
    options=['season', 'at-bats'],
    description='compare by:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check')
   
def switch_vals(arrA, arrB):
    temp = copy.copy(arrA)
    arrA = arrB
    arrB = temp
    return arrA, arrB

def on_button_bat1(change):
    # TODO: This function is rather terrible, but abstracting seems like a big pain right now.
    # Switch Arlo lines
    dx0 = np.diff(arlo_lines[0].x)[0]
    dy0 = np.diff(arlo_lines[0].y)[0]
    
    dx1 = np.diff(arlo_lines[1].x)[0]
    dy1 = np.diff(arlo_lines[1].y)[0]
    
    if change['new'] == 'at-bats':
        arlo_lines[1].x = [0, dx1]
        arlo_lines[1].y = [0, dy1]

        arlo_lines[0].x = [dx1, dx0 + dx1]
        arlo_lines[0].y = [dy1, dy0 + dy1]
        
    elif change['new'] == 'season':
        arlo_lines[0].x = [0, dx0]
        arlo_lines[0].y = [0, dy0]

        arlo_lines[1].x = [dx0, dx0 + dx1]
        arlo_lines[1].y = [dy0, dy0 + dy1]
    else:
        print(change)
    
    # Move second vert line
    if change['new'] == 'at-bats':
        vert_lines[1].x = x=2*[atbatscum[0, 1]]
    elif change['new'] == 'season':
        vert_lines[1].x = x=2*[atbatscum[1, 1]]
    else:
        print(change)
    
button_bat1.observe(on_button_bat1, names='value')

box = widgets.GridspecLayout(3, 4, layout=widgets.Layout(width='80%', height='400px'))

#leg = plt.legend() 
box[:, 0:3] = fig_batting
box[1, 3] = button_bat1
box

GridspecLayout(children=(Figure(animation_duration=1000, axes=[Axis(label='at-bats', scale=LinearScale(max=110…

In this graph we show runs vs at-bats - this means that the slope of each line represents the batting average for that time period.
The solid line represents the first season and the dashed line represents the second season.

When we compare the two players by season, we see that Arlo has a better BA each year (greater slope).
Yet somehow Jack still comes out the overall leader in BA.

Now let's try looking at the same data, but sort by at-bats.
From this vantage, Arlo still leads in one group but Jack leads in the other.
Since the leadership is now mixed, it should be no surprise that either player might attain the higher overall BA.

Notice that Arlo has a significantly better BA in the small group (20 at-bats).
However, Jack's advantage in the large group (80 at-bats) represents the dominant piece of this puzzle.
Even though Jack's BA lead is less dramatic, his lead is drawn out over a greater extent (number of at-bats).

# Interactive Challenge!

Now consider two other baseball players: Joe and Kamala.
Imagine they have played for three seasons.
Can you design a scenario where Joe has a better BA for each season, but Kamala has the better overall BA?

In [115]:
# Make interactive batting challenge

# TODO: Make a toggle or second challenge or just quiz question where the at-bats for each section is the same for both players.
joe_hits_cum = np.array([0, 10, 20, 30])
joe_atbats_cum = np.array([0, 20, 40, 60])

kamala_hits_cum = np.array([0, 12, 24, 36])
kamala_atbats_cum = np.array([0, 20, 40, 60])

joe_lineA = plt.plot(x=joe_atbats_cum[0:2], y=joe_hits_cum[0:2], colors=['blue'], stroke_width=5)
joe_lineB = plt.plot(x=joe_atbats_cum[1:3], y=joe_hits_cum[1:3], colors=['blue'], stroke_width=5, line_style='dashed')
joe_lineC = plt.plot(x=joe_atbats_cum[2:4], y=joe_hits_cum[2:4], colors=['blue'], stroke_width=5, line_style='dotted')
kamala_lineA = plt.plot(x=kamala_atbats_cum[0:2], y=kamala_hits_cum[0:2], colors=['red'], stroke_width=5)
kamala_lineB = plt.plot(x=kamala_atbats_cum[1:3], y=kamala_hits_cum[1:3], colors=['red'], stroke_width=5, line_style='dashed')
kamala_lineC = plt.plot(x=kamala_atbats_cum[2:4], y=kamala_hits_cum[2:4], colors=['red'], stroke_width=5, line_style='dotted')

joe_line_overall = plt.plot(x=joe_atbats_cum[[0, -1]], y=joe_hits_cum[[0, -1]], colors=['lightblue'], stroke_width=3)
kamala_line_overall = plt.plot(x=kamala_atbats_cum[[0, -1]], y=kamala_hits_cum[[0, -1]], colors=['pink'], stroke_width=3)

joe_scatter0 = plt.scatter(x=[joe_atbats_cum[0]], y=[joe_hits_cum[0]], colors=['blue'], default_size=200, enable_move=False)
kamala_scatter0 = plt.scatter(x=[kamala_atbats_cum[0]], y=[kamala_hits_cum[0]], colors=['red'], default_size=200, enable_move=False)

joe_scatter = plt.scatter(x=joe_atbats_cum[1:], y=joe_hits_cum[1:], colors=['blue'], default_size=200, enable_move=True)
kamala_scatter = plt.scatter(x=kamala_atbats_cum[1:], y=kamala_hits_cum[1:], colors=['red'], default_size=200, enable_move=True)

status_text_congratulations = plt.label([""], x=[30], y=[20], 
                            align='middle', font_weight='bold', default_size=24, colors=['Black'])

htmltext = "blah blah"
htmlWidget = widgets.HTML(value = f"<b><font color='red'>{htmltext}</b>")

def on_joe_move(change):
    if change['name'] == 'x':
        newxs = change['new']
        allnewxs = np.insert(newxs, 0, joe_scatter0.x[0])
        
        if np.any(np.diff(allnewxs) < 0):
            # Reject the move
            joe_scatter.x = change['old']
            return
        
        joe_lineA.x = allnewxs[0:2]
        joe_lineB.x = allnewxs[1:3]
        joe_lineC.x = allnewxs[2:4]
        joe_line_overall.x = allnewxs[[0, -1]]
        
    if change['name'] == 'y':
        newys = change['new']
        allnewys = np.insert(newys, 0, joe_scatter0.y[0])
        
        if np.any(np.diff(allnewys) < 0):
            # Reject the move
            joe_scatter.y = change['old']
            return
        
        joe_lineA.y = allnewys[0:2]
        joe_lineB.y = allnewys[1:3]
        joe_lineC.y = allnewys[2:4]
        joe_line_overall.y = allnewys[[0, -1]]

    is_simpson_bat(joe_lineA, joe_lineB, joe_lineC, kamala_lineA, kamala_lineB, kamala_lineC)
    
def on_kamala_move(change):
    if change['name'] == 'x':
        newxs = change['new']
        allnewxs = np.insert(newxs, 0, kamala_scatter0.x[0])

        if np.any(np.diff(allnewxs) < 0):
            # Reject the move
            kamala_scatter.x = change['old']
            return
        
        kamala_lineA.x = allnewxs[0:2]
        kamala_lineB.x = allnewxs[1:3]
        kamala_lineC.x = allnewxs[2:4]
        kamala_line_overall.x = allnewxs[[0, -1]]
        
    if change['name'] == 'y':
        newys = change['new']
        allnewys = np.insert(newys, 0, kamala_scatter0.y[0])
        
        if np.any(np.diff(allnewys) < 0):
            # Reject the move
            kamala_scatter.y = change['old']
            return
        
        kamala_lineA.y = allnewys[0:2]
        kamala_lineB.y = allnewys[1:3]
        kamala_lineC.y = allnewys[2:4]
        kamala_line_overall.y = allnewys[[0, -1]]
    
    is_simpson_bat(joe_lineA, joe_lineB, joe_lineC, kamala_lineA, kamala_lineB, kamala_lineC)

def is_slope_greater(lineA, lineB):
    slopeA = np.diff(lineA.y) / np.diff(lineA.x)
    slopeB = np.diff(lineB.y) / np.diff(lineB.x)
    if slopeA > slopeB:
        return True
    else:
        return False
    
def is_simpson_bat(joe_lineA, joe_lineB, joe_lineC, kamala_lineA, kamala_lineB, kamala_lineC):
    
    conds = [is_slope_greater(joe_lineA, kamala_lineA), 
             is_slope_greater(joe_lineB, kamala_lineB), 
             is_slope_greater(joe_lineC, kamala_lineC), 
             is_slope_greater(joe_line_overall, kamala_line_overall)]
    
    if conds[0]:
        nameA = 'Joe'
    else:
        nameA = 'Kamala'
        
    if conds[1]:
        nameB = 'Joe'
    else:
        nameB = 'Kamala'
        
    if conds[2]:
        nameC = 'Joe'
    else:
        nameC = 'Kamala'
        
    if conds[3]:
        name_overall = 'Joe'
    else:
        name_overall = 'Kamala'
    
    htmlWidget.value = "<b><font color='black'>" + \
    """
    <table style='width:250px;font-size:2em'>
      <tr>
        <th>Year</th>
        <th>Leader</th>
      </tr>
      <tr>
        <td>2020</td>
        <td>{}</td>
      </tr>
      <tr>
        <td>2021</td>
        <td>{}</td>
      </tr>
      <tr>
        <td>2022</td>
        <td>{}</td>
      </tr>
      <tr>
        <td>Overall</td>
        <td>{}</td>
      </tr>
    </table>
    """.format(nameA, nameB, nameC, name_overall) + "</b>"
    
    if conds[0] and conds[1] and conds[2] and not conds[3]:
        status_text_congratulations.text = ['Congratulations!']
        
joe_scatter.observe(on_joe_move, names=['x', 'y'])
kamala_scatter.observe(on_kamala_move, names=['x', 'y'])
    
fig_interactive_batting = plt.figure(marks=[joe_lineA, joe_lineB, joe_lineC, 
                                            kamala_lineA, kamala_lineB, kamala_lineC, 
                                            joe_line_overall, kamala_line_overall,
                                            joe_scatter0, kamala_scatter0, 
                                            joe_scatter, kamala_scatter, 
                                            status_text_congratulations],
                        layout=widgets.Layout(width='auto', height='auto'), animation_duration=100, 
                        fig_margin={'top':20, 'bottom':50, 'left':50, 'right':20}, display_legend=True)

is_simpson_bat(joe_lineA, joe_lineB, joe_lineC, kamala_lineA, kamala_lineB, kamala_lineC)

box_interactive_batting = widgets.GridspecLayout(3, 4, layout=widgets.Layout(width='80%', height='400px'))

box_interactive_batting[:, 0:3] = fig_interactive_batting
box_interactive_batting[1, 3] = htmlWidget

box_interactive_batting

GridspecLayout(children=(Figure(animation_duration=100, fig_margin={'top': 20, 'bottom': 50, 'left': 50, 'righ…

# QUIZ: something

# Upload data files
<p class="lead">This <a href="https://jupyter.org/">Jupyter notebook</a>
shows how to upload data files to be converted
to [Photon-HDF5](http://photon-hdf5.org) format. </p>

<i>Please send feedback and report any problems to the 
[Photon-HDF5 google group](https://groups.google.com/forum/#!forum/photon-hdf5).</i>

<br>
<div class="alert alert-warning">
<b>NOTE</b> Uploading data files is only necessary when running the notebook online.
</div>
<div class="alert-warning">
<b>NOTE</b> Uploading data files is only necessary when running the notebook online.
</div>
<div class="alert">
<b>NOTE</b> Uploading data files is only necessary when running the notebook online.
</div>
<b>NOTE</b> Uploading data files is only necessary when running the notebook online.

# Example 3: Kind, rich men.

TODO: CHOOSE Map onto this example or cut and stick with shapes.

In this example, a woman seeks a husband.
She desires a man who is rich and kind.
She divides her search for this man into two groups, bald men and men with hair.

She finds:
The proportion of rich men is higher in the "hair" group than the "bald" group.
The proportion of kind men is higher in the "hair" group than the "bald" group.
However, surprisingly, the proportion of (rich AND kind) men is higher in the "bald" group than the "hair" group.

How can we explain this?

Let's look at the data.

In [ ]:
# make shape data
shape_df = pd.DataFrame({'row':[0, 0, 0, 0, 0, 
                              1, 1, 1, 1, 1], 
                    'column':[0, 1, 2, 3, 4, 
                              0, 1, 2, 3, 4],
                     'shape':[1, 1, 0, 0, 0, 
                              0, 0, 1, 1, 1], 
                     'color':[0, 0, 0, 1, 1, 
                              0, 0, 1, 1, 1]})

shape_df['color'] = shape_df['color'].map({0:'red', 1:'blue'})

def shuffle_tokens(shape_df):
    # Shuffle within each row
    # There are ways to permute within pandas, but they seemed gross.
    shuffled_inds = list(np.random.permutation(range(0, 5))) + list(np.random.permutation(range(5, 10)))
    shape_df['shape'] = shape_df['shape'].iloc[shuffled_inds].to_numpy()
    shape_df['color'] = shape_df['color'].iloc[shuffled_inds].to_numpy()
    return shape_df

#shape_df = shuffle_tokens(shape_df)
#shape_df

In [ ]:
# TODO: fix the shuffle - not sure what's wrong

# simpson's paradox with tokens
x_sc2 = bq.LinearScale(min=-0.5, max=4.5)
y_sc2 = bq.LinearScale(min=0, max=1)

x_ax2 = bq.Axis(label='length', scale=x_sc2)
y_ax2 = bq.Axis(label='height', scale=y_sc2, orientation='vertical')

# shape choices
sA = 'triangle-up'
sB = 'cross'
sC = 'square'
    
def make_simpsons_shape_box(shape_df):
    shapesA_df = shape_df[shape_df['shape']==0]
    shapesB_df = shape_df[shape_df['shape']==1]
    shapesA = bq.Scatter(x=shapesA_df['column'], y=shapesA_df['row'], colors=list(shapesA_df['color']), 
                          marker=sA, default_size=500, scales={'x': x_sc2, 'y': y_sc2})
    shapesB = bq.Scatter(x=shapesB_df['column'], y=shapesB_df['row'], colors=list(shapesB_df['color']), 
                          marker=sB, default_size=500, scales={'x': x_sc2, 'y': y_sc2})
    div_line = bq.Lines(x=[-1, 5], y=[0.5, 0.5], colors=['black'], stroke_width=3, scales={'x': x_sc2, 'y': y_sc2})
    fig_shape = plt.figure(marks=[shapesA, shapesB, div_line], axes=[], animation_duration=1000, 
                           layout=widgets.Layout(width='auto', height='200px', border='3px solid black'), 
                          fig_margin={'top':20, 'bottom':20, 'left':20, 'right':20})
    
    button_show = widgets.ToggleButtons(
        value='shape',
        options=['nothing', 'shape', 'color', 'shape AND color'],
        description='show me:',
        disabled=False,
        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Description',
        icon='check')
    
    button_shuffle = widgets.Button(
        description='Shuffle now',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Description')
    
    button_shuffle_bool = widgets.Checkbox(
            value=True,
            description='Shuffle on observation',
            disabled=False
        )
    
    def on_button_show(change, shapesA, shapesB):
        
        if box[3,0].value: # shuffle check box
            on_button_shuffle(change, shape_df)
    
        shapesA_df = shape_df[shape_df['shape']==0]
        shapesB_df = shape_df[shape_df['shape']==1]
    
        if change['new'] == 'nothing':

            shapesA.marker='square'
            shapesB.marker='square'
            shapesA.colors=['gray']
            shapesB.colors=['gray']
            
        elif change['new'] == 'shape':

            shapesA.marker='triangle-up'
            shapesB.marker='cross'
            shapesA.colors=['gray']
            shapesB.colors=['gray']
            
        elif change['new'] == 'color':

            shapesA.marker='square'
            shapesB.marker='square'
            shapesA.colors=list(shapesA_df['color'])
            shapesB.colors=list(shapesB_df['color'])
            
        elif change['new'] == 'shape AND color':

            shapesA.marker='triangle-up'
            shapesB.marker='cross'
            shapesA.colors=list(shapesA_df['color'])
            shapesB.colors=list(shapesB_df['color'])

        else:
            raise

    def on_button_shuffle(change, shape_df):
        shape_df = shuffle_tokens(shape_df)
        shapesA_df = shape_df[shape_df['shape']==0]
        shapesB_df = shape_df[shape_df['shape']==1]
        
        shapesA.x = shapesA_df['column']
        shapesA.y = shapesA_df['row']
        shapesB.x = shapesB_df['column']
        shapesB.y = shapesB_df['row']
        
    button_shuffle.on_click(lambda change:on_button_shuffle(change, shape_df))
    button_show.observe(lambda change: on_button_show(change, shapesA, shapesB), 'value')

    box = widgets.GridspecLayout(4, 4, grid_gap='0px', width='100%')
    
    box[0:2, 0] = button_show
    box[2, 0] = button_shuffle
    box[3, 0] = button_shuffle_bool
    
    box[:, 1:] = fig_shape

    return shapesA, shapesB, box


shapesA, shapesB, box = make_simpsons_shape_box(shape_df)
box[0,0].value='nothing'
box

# Red Triangles

There are ten objects divided into two groups of five.
Each object has a color (red or blue) and a shape (cross or triangle).
Initially, both the color and shape are unknown to us.

**Our goal is to find the group (upper/lower) with the most red triangles.**

### Shape:
First, click on "shape" to discover the shape of each object.

Q: What is the probability of finding a triangle when choosing from the top group?
(answer: 2/5)

Q: Similarly, what is the probability of finding a triangle when choosing from the bottom group?
(answer: 3/5)

Q: Which group should you choose from to ?
(answer: bottom)

### Color:
Now click on "color" to observe the colors.

Q: What is the probability of finding a red object when choosing from the top group?
(answer: 3/5)

Q: Similarly, what is the probability of finding a red object when choosing from the bottom group?
(answer: 2/5)

Q: Which group should you choose from?
(answer: bottom)

### Shape AND Color:
Q: Before you click on the last button, guess which group will have the most red triangles. (What is your reasoning?)

Now select "shape AND color".
Q: What is the probability of finding a red triangle in the top group?
(answer: 2/5)

Q: In the bottom?
(answer: 1/5)

Q: Which group should you choose from? (Was your guess correct?)
(answer: top)

## Review:
Let's try to understand what is going on here.
While there are more triangles on top and there are more red objects on top, the greatest proportion of red triangles is on the bottom.
This is because of how the two properties are correlated.

On the top, redness is highly correlated with triangleness.
On the bottom, redness is actually anti-correlated with triangleness.

## Wrap up

### Fish

In the fish example, the fish had three independent properties: length, width, and color.

We divided the fish into two subgroups based on one property - color.
We found that a correlation between length and width found within each color subgroup.
We also found that this correlation could be reversed when we aggregated color subgroups.

### Men

In the marriage example, the men had three independent properties: richness, kindness, and hairiness. 

We found a correlation between richness and hairiness.
We found a correlation between kindness and hairiness.
We found that when we create a composite property (rich AND kind) that this property is correlated with hairiness, but in the opposite sense.


## Complicated example

Show one of the canonical examples, e.g. the effect of a drug in a table.

Lots of numbers.
Somehow, "adding up" small things results in a larger result than adding up large things.
This is Simpson's paradox.

You can check these numbers to confirm the story - a pain, but straightforward.

Clearly this is *important* (we want to know which drugs to take and we want our doctors to know).

- Show the number table
- Compute for them how simpsons paradox arises (they don't have to do any math, but they can believe it).
- Question 1: Choose the sentence that explains ..? the source of the paradox?
- Question 2: Which drug should you choose? A - #1, B - #2, C - either, D - depends on if you are a man or woman

TODO: can we decouple this example from gender?

### Highlight how this notebook addresses the Brilliant Teaching Principles (https://brilliant.org/principles/):

1. Excites: The greatest challenges to education are disinterest and apathy.

Fish are kind of silly looking.
TODO: Maybe move in that direction for the shape example or at least for the story.

2. Cultivates curiosity: Questions and storytelling that cultivate natural curiosity are better than the threat of a test.

No threats here.
Incorporate a range of difficulty including some very beginner.
Also, the interactive "games" have no time limit - this deemphasizes success vs failure, certainly in a test sense.
    
3. Is active: Effective learning is active, not passive. Watching a video is not enough.

Fish graphs develop along with the story.
Questions interspersed maintain engagement.

Interactive game is active.

4. Is applicable: Use it or lose it: it is essential to apply what you're learning as you learn it.

Drug example is very applicable.
Making decisions (or even just understanding your doctor) in the face of statistics is a valuable skill.

5. Is community driven: A community that challenges and inspires you is invaluable.

Not sure how to incorporate this.

6. Doesn't discriminate: Your age, country, and gender don't determine what you are capable of learning.

I took care to choose examples that seemed accessible to a wide audience.
Fish are innocuous.
Simple shapes and colors are accessible across grades, languages, cultures.
Simpson examples often include gender - something I avoided for this reason.

7. Allows for failure: The best learners allow themselves to make many mistakes along their journey.

TODO: Offer more opportunity for mistakes (that can be then fixed).

8. Sparks questions: The culmination of a great education isn't knowing all the answers — it's knowing what to ask.

TODO: End with some good food for thought. Maybe one mathy one and one practical.
